<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ
</h1>

---

<h1 style="text-align: center;">
    Перцептрон Розенблатта
</h1>

В данном ноутбуке мы:  

* реализуем класс `Perceptron` - нейрон с пороговой функцией активации

* обучим и протестируем перцептрон на сгенерированных и реальных данных (файлы с реальными данными помещены в папку `data` в этой же директории)

* сравним качество работы вашего класса с классом из библиотеки `scikit-learn` (`sklearn.linear_model.Perceptron`)

## Введение

Почти любой алгоритм машинного обучения, решающий задачу **классификации** или **регрессии**, работает так:

1. **Стадия инициализации**: задаются его **гиперпараметры**, то есть те величины, которые не "выучиваются" алгоритмом в процессе обучения самостоятельно.

2. **Стадия обучения**: алгоритм запускается на данных, **обучаясь** на них и меняя свои **параметры** (не путать с *гипер*параметрами) каким-то определённым образом (например, с помощью *метода градиентного спуска* или *метода коррекции ошибки*), исходя из функции потерь (её называют *loss function*). Функция потерь, по сути, говорит, где и как ошибается модель.

3. **Стадия предсказания**: модель готова, и теперь с её помощью можно делать **предсказания** на новых объектах.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline

## Класс `Perceptron`

В даном разделе будет решаться задача **бинарной классификации** с помощью перцептрона:

* **Входные данные**: матрица $X$ размера $(n, m)$ и столбец $y$ из нулей и единиц размера $(n, 1)$. Строкам матрицы соответствуют объекты, столбцам - признаки (то есть строка $i$ есть набор признаков (**признаковое описание**) объекта $X_i$).

* **Выходные данные**: столбец $\hat{y}$ из нулей и единиц размера $(n, 1)$ - предсказания алгоритма.

Чтобы понять, как мы будем обновлять параметры модели (веса), нужно знать, какую функцию потерь мы оптимизируем. В данном случае мы решаем задачу бинарной классификации ($2$ класса: $1$ или $0$), возьмём в качестве функции потерь среднеквадратичную ошибку:  

$$
Loss(\hat{y}, y) = \frac{1}{2n}\sum_{i=1}^{n} (\hat{y_i} - y_i)^2 = \frac{1}{2n}\sum_{i=1}^{n} (f(w \cdot X_i + b) - y_i)^2
$$  

Здесь $w \cdot X_i$ - скалярное произведение, а $f$ - пороговая функция: 

$$
f(x) =
\begin{cases}
1, &\text{если } x > 0 \\
0, &\text{если } x \le 0
\end{cases}
$$  

**Примечание.** На самом деле можно считать, что $b$ - свободный член - является частью вектора весов $w_0$, приписав к $X$ слева единичный столбец. Тогда в скалярном произведении с каждым объектом $b$ будет именно как свободный член. При реализации класса `Perceptron` мы будем обновлять $b$ отдельно от $w$.

Реализуем функцию потерь:

In [ ]:
def loss(y_pred, y):
    """
    Считаем среднеквадратичную ошибку
    """

    return 0.5 * np.mean((y_pred - y) ** 2)

Поскольку у пороговой функции не существует производной в нуле, то мы не можем использовать градиентный спуск, ведь:  

$$
\frac{\partial Loss}{\partial w} = \frac{1}{n} X^T\left(f(w \cdot X) - y\right)f'(w \cdot X)
$$  

где $f^{'}(w \cdot X)$ в точке $0$ посчитать не получится. Но хочется как-то обновлять веса, иначе обучения не случится.

Поэтому предлагается обновлять так:   

$$
w^{j+1} = w^{j} - \alpha\Delta{w^{j}}
$$ 

$$
b^{j+1} = b^{j} - \alpha\Delta{b^{j}}
$$ 

где:  

$$
\Delta{w} = \frac{1}{n}X^T(\hat{y} - y) = \frac{1}{n}X^T(f(X \cdot w^j + b^j) - y)
$$  

$$
\Delta{b} = \frac{1}{n}X^T(\hat{y} - y) = \frac{1}{n}1^T(f(X \cdot w^j + b^j) - y)
$$  

где $w \cdot X$ - матричное произведение столбца весов $w$ на матрицу объектов-признаков $X$, $1^T$ - вектор-строка из единиц, а индекс $j$ - номер итерации градиентного спуска.

Это правило является неким частным случаем градиентного спуска для данного случая (см. [правило Хебба](https://ru.wikipedia.org/wiki/Дельта-правило), [метод коррекции ошибки](https://ru.wikipedia.org/wiki/Метод_коррекции_ошибки)).

Вооружившись всеми формулами, напишем свой класс `Perceptron`. 

**Примечание.** В коде ниже `y_pred` - это $\hat{y}$ из формул выше.

In [ ]:
class Perceptron:
    def __init__(self, w=None, b=0):
        """
        param w - вектор весов
        param b - смещение
        """

        # пока что мы не знаем размер матрицы X, а значит не знаем, сколько будет весов
        self.w = w
        self.b = b
        
    def activate(self, x):
        return (x > 0).astype(int)
        
    def forward_pass(self, X):
        """
        Эта функция рассчитывает ответ перцептрона при предъявлении набора объектов
        param X - матрица объектов размера (n, m), каждая строка - отдельный объект
        return - вектор размера (n,) из нулей и единиц с ответами перцептрона 
        """

        # y_pred == y_predicted - предсказанные классы
        y_pred = self.activate(X @ self.w + self.b)
        return y_pred
    
    def backward_pass(self, X, y, y_pred, lr=0.005):
        """
        Обновляет значения весов перцептрона в соответствии с этим объектом
            param X - матрица объектов размера (n, m)
            param y - вектор правильных ответов размера (n,)
            param lr - "скорость обучения" (символ alpha в формулах выше)
        
        В этом методе ничего возвращать не нужно, только правильно поменять веса
        с помощью градиентного спуска.
        """
        
        n = X.shape[0]
        
        self.w -= lr / n * X.T @ (y_pred - y)
        self.b -= lr * np.mean(y_pred - y)
    
    def fit(self, X, y, num_epochs=300):
        """
        Спускаемся в минимум
            param X - матрица объектов размера (n, m)
            param y - вектор правильных ответов размера (n,)
            param num_epochs - количество итераций обучения
            return - вектор значений функции потерь
        """
        
        # вектор нулей размера (m,)
        self.w = np.zeros(X.shape[1])
        # смещение
        self.b = 0
        # значения функции потерь на различных итерациях обновления весов
        losses = []
        
        for i in range(num_epochs):
            # предсказания с текущими весами
            y_pred = self.forward_pass(X)
            # считаем функцию потерь с текущими весами
            losses.append(loss(y_pred, y))
            # обновляем веса в соответсвии с тем, где ошиблись раньше
            self.backward_pass(X, y, y_pred)

        return losses

Класс готов. Посмотрим, правильно ли ведёт себя наш перцептрон. Далее идут несколько ячеек с тестовым кодом, вам нужно просто запустить их и проверить, чтобы результаты запуска совпадали с соответствующими числами из таблиц.

#### Проверка `forward_pass`

In [ ]:
w = np.array([1., 2.])
b = 2.
X = np.array([[1., 3.], [2., 4.], [-1., -3.2]])

perceptron = Perceptron(w, b)
y_pred = perceptron.forward_pass(X)
print(y_pred)

#### Проверка `backward_pass`

In [ ]:
y = np.array([1, 0, 1])

In [ ]:
perceptron.backward_pass(X, y, y_pred)

print(perceptron.w)
print(perceptron.b)

Посмотрим, как меняется функция потерь в течение процесса обучения на реальных данных - датасет "Яблоки и Груши".

In [ ]:
data = pd.read_csv('data/apples_pears.csv')

In [ ]:
data.head()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(data.iloc[:, 0], data.iloc[:, 1], c=data['target'], cmap='rainbow')
plt.title('Яблоки и груши', fontsize=15)
plt.xlabel('Симметричность', fontsize=14)
plt.ylabel('Желтизна', fontsize=14)
plt.show()

**Вопрос.** Какой класс соответствует яблокам (какого они цвета на графике)?

Обозначим, что здесь признаки, а что - классы:

In [ ]:
# матрица объекты-признаки
X = data.iloc[:, :2].values
# классы (столбец из нулей и единиц)
y = data['target'].values

**Вывод функции потерь.** Функция потерь должна убывать и в итоге стать близкой к $0$.

In [ ]:
# может вызывать баги на старых версиях jupyter notebook
# %%time

perceptron = Perceptron()
losses = perceptron.fit(X, y)

plt.figure(figsize=(10, 8))
plt.plot(losses)
plt.title('Функция потерь', fontsize=15)
plt.xlabel('Номер итерации', fontsize=14)
plt.ylabel('$Loss(\hat{y}, y)$', fontsize=14)
plt.show()

Посмотрим, как перцептрон классифицировал объекты из выборки:

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(data.iloc[:, 0], data.iloc[:, 1], c=perceptron.forward_pass(X).ravel(), cmap='spring')
plt.title('Яблоки и груши', fontsize=15)
plt.xlabel('Симметричность', fontsize=14)
plt.ylabel('Желтизна', fontsize=14)
plt.show()

Справился идеально. Однако просьба обратить внимание, что это очень простая, **линейно разделимая**, выборка.

## Предсказание пола по голосу

Сравним качество работы нашего перцептрона и алгоритма из библиотеки `sklearn` на датасете с сайта [Kaggle](https://www.kaggle.com) - [Gender Recognition by Voice](https://www.kaggle.com/primaryobjects/voicegender). В данном датасете в качестве признаков выступают различные звуковые характеристики голоса, а в качестве классов - пол (мужчина/женщина). Подробнее о самих признаках можно почитать [на странице датасета](https://www.kaggle.com/primaryobjects/voicegender) (на английском). Нашей целью пока что является просто протестировать на этих данных два алгоритма.

In [ ]:
import pandas as pd
from sklearn.linear_model import Perceptron as skPerceptron
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv('data/voice.csv')
data['label'] = data['label'].apply(lambda x: 1 if x == 'male' else 0)

In [ ]:
data.head()

In [ ]:
# перемешаем данные, изначально там сначала идут все мужчины, потом все женщины
data = data.sample(frac=1)

In [ ]:
# матрица объекты-признаки
X_train = data.iloc[:int(len(data) * 0.7), :-1].values
# истинные значения пола (мужчина/женщина)
y_train = data.iloc[:int(len(data) * 0.7), -1].values

# матрица объекты-признаки
X_test = data.iloc[int(len(data)*0.7):, :-1].values
# истинные значения пола (мужчина/женщина)
y_test = data.iloc[int(len(data)*0.7):, -1].values

Натренируем наш перцептрон и перцептрон из `sklearn` на этих данных:

In [ ]:
RANDOM_SEED = 42

perceptron = Perceptron()
perceptron.fit(X_train, y_train)

sk_perceptron = skPerceptron(random_state=RANDOM_SEED)
sk_perceptron.fit(X_train, y_train)

Сравним доли правильных ответов на тестовых данных:

In [ ]:
print(f'Точность нашего перцептрона: {accuracy_score(y_test, perceptron.forward_pass(X_test)) * 100:.3f}%')
print(f'Точность перцептрона из sklearn: {accuracy_score(y_test, sk_perceptron.predict(X_test)) * 100:.3f}%')

Попробуем поставить число итераций побольше:

In [ ]:
RANDOM_SEED = 42

perceptron = Perceptron()
perceptron.fit(X_train, y_train, num_epochs=5000)

sk_perceptron = skPerceptron(random_state=RANDOM_SEED, max_iter=5000)
sk_perceptron.fit(X_train, y_train)

In [ ]:
print(f'Точность нашего перцептрона: {accuracy_score(y_test, perceptron.forward_pass(X_test)) * 100:.3f}%')
print(f'Точность перцептрона из sklearn: {accuracy_score(y_test, sk_perceptron.predict(X_test)) * 100:.3f}%')

**Вопрос:** Хорошее ли качество показывает перцептрон? Как вы думаете, почему?

**Подсказка.** Попробуйте нормализовать данные.

## Важно

Стоит понимать, что перцептрон сам по себе не используется в приложениях. Мы продемонстрровали его вам, чтобы вы знали, с чего всё начиналось. На самом деле это просто один нейрон с пороговой функцией активации, который не используется в многослойных нейросетях и каких-либо прикладных задачах, но всё же является хорошим учебным примером, помогающим понять то, как обновляются веса в соответствии с ошибками и перейти к рассмотрению более полезных моделей (нейронов с другими функциями активации).

## Полезные ссылки

1. Lecture Notes Стэнфордского университета - http://cs231n.github.io/neural-networks-1/

2. Википедия про перцептрон - https://ru.wikipedia.org/wiki/Перцептрон